# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

### ノック41：データを読み込んで利用データを整形しよう

In [1]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [3]:
year_months = list(uselog_months['年月'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['年月'] == year_months[i]]
    tmp.rename(columns={'count': 'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['年月'] == year_months[i-1]]
    del tmp_before['年月']
    tmp_before.rename(columns={'count': 'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/2759125647.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.rename(columns={'count': 'count_0'}, inplace=True)
/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/2759125647.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_before.rename(columns={'count': 'count_1'}, inplace=True)
/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/2759125647.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [5]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer['is_deleted'] ==1]
exit_customer['exit_date'] = pd.to_datetime('20180401')
exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
for i in range(len(exit_customer)):
    exit_customer['exit_date'].iloc[i] = exit_customer['end_date'].iloc[i] - relativedelta(months=1)
exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')
uselog['年月']  = uselog['年月'].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on =['customer_id', '年月'], how='left')
print(len(uselog))
exit_uselog.head()

/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/3856478467.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['exit_date'] = pd.to_datetime('20180401')
/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/3856478467.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['end_date'] = pd.to_datetime(exit_customer['end_date'])
/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/3856478467.py:6: SettingWithCopyWarning: 
A value is trying to be

33851


/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/3856478467.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exit_customer['年月'] = exit_customer['exit_date'].dt.strftime('%Y%m')


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [6]:
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


### ノック43：継続顧客のデータを作成しよう

In [7]:
conti_customer  = customer.loc[customer['is_deleted'] ==0]
conti_uselog= pd.merge(uselog, conti_customer, on=['customer_id'], how='left')
print(len(conti_customer))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

2842
27422


In [8]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201902,OA468240,4,5.0,XXXX,C02,F,2017-12-01,NaN,CA3,...,デイタイム,7500.0,入会費無料,6.416667,6.0,10.0,4.0,1.0,2019-04-30,16.0
1,201807,HI615448,9,NaN,XXXXX,C01,M,2018-07-05,NaN,CA1,...,オールタイム,10500.0,通常,6.777778,7.0,9.0,4.0,1.0,2019-04-30,9.0
2,201902,HI196175,5,5.0,XXX,C01,M,2015-11-01,NaN,CA1,...,オールタイム,10500.0,通常,4.666667,5.0,6.0,3.0,1.0,2019-04-30,41.0
3,201811,HD315748,3,6.0,XXXXX,C01,M,2017-04-01,NaN,CA1,...,オールタイム,10500.0,通常,5.250000,6.0,8.0,2.0,1.0,2019-04-30,24.0
4,201808,PL288896,11,NaN,XXXXXX,C01,F,2018-08-11,NaN,CA1,...,オールタイム,10500.0,通常,8.375000,8.0,12.0,6.0,1.0,2019-04-30,8.0


In [9]:
predict_date = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_date))
predict_date.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201902,OA468240,4,5.0,XXXX,C02,F,2017-12-01,NaN,CA3,...,7500.0,入会費無料,6.416667,6.0,10.0,4.0,1.0,2019-04-30,16.0,NaT
1,201807,HI615448,9,NaN,XXXXX,C01,M,2018-07-05,NaN,CA1,...,10500.0,通常,6.777778,7.0,9.0,4.0,1.0,2019-04-30,9.0,NaT
2,201902,HI196175,5,5.0,XXX,C01,M,2015-11-01,NaN,CA1,...,10500.0,通常,4.666667,5.0,6.0,3.0,1.0,2019-04-30,41.0,NaT
3,201811,HD315748,3,6.0,XXXXX,C01,M,2017-04-01,NaN,CA1,...,10500.0,通常,5.250000,6.0,8.0,2.0,1.0,2019-04-30,24.0,NaT
4,201808,PL288896,11,NaN,XXXXXX,C01,F,2018-08-11,NaN,CA1,...,10500.0,通常,8.375000,8.0,12.0,6.0,1.0,2019-04-30,8.0,NaT


### ノック44：予測する月の在籍期間を作成しよう

In [12]:
predict_date['period'] = 0
predict_date['now_date'] =pd.to_datetime(predict_date['年月'], format='%Y%m')
predict_date['start_date'] = pd.to_datetime(predict_date['start_date'])
for i in range(len(predict_date)):
    delta = relativedelta(predict_date['now_date'][i], predict_date['start_date'][i])
    predict_date['period'][i] = int(delta.years * 12 + delta.months)
predict_date.head()

/var/folders/tl/3mcdykwx5g145dz_cpc1vdxc0000gn/T/ipykernel_70646/981516963.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  predict_date['period'][i] = int(delta.years * 12 + delta.months)


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201902,OA468240,4,5.0,XXXX,C02,F,2017-12-01,NaN,CA3,...,6.416667,6.0,10.0,4.0,1.0,2019-04-30,16.0,NaT,14,2019-02-01
1,201807,HI615448,9,NaN,XXXXX,C01,M,2018-07-05,NaN,CA1,...,6.777778,7.0,9.0,4.0,1.0,2019-04-30,9.0,NaT,0,2018-07-01
2,201902,HI196175,5,5.0,XXX,C01,M,2015-11-01,NaN,CA1,...,4.666667,5.0,6.0,3.0,1.0,2019-04-30,41.0,NaT,39,2019-02-01
3,201811,HD315748,3,6.0,XXXXX,C01,M,2017-04-01,NaN,CA1,...,5.250000,6.0,8.0,2.0,1.0,2019-04-30,24.0,NaT,19,2018-11-01
4,201808,PL288896,11,NaN,XXXXXX,C01,F,2018-08-11,NaN,CA1,...,8.375000,8.0,12.0,6.0,1.0,2019-04-30,8.0,NaT,0,2018-08-01


### ノック45：欠損値を除去しよう

In [13]:
predict_date.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               255
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [14]:
predict_date = predict_date.dropna(subset=['count_1'])
predict_date.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2639
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2639
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [15]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_date = predict_date[target_col]
predict_date.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,入会費無料,デイタイム,F,5.0,1.0,14,0.0
2,通常,オールタイム,M,5.0,1.0,39,0.0
3,通常,オールタイム,M,6.0,1.0,19,0.0
5,通常,オールタイム,M,10.0,1.0,2,0.0
6,通常,デイタイム,M,6.0,1.0,4,0.0


In [16]:
predict_date  =pd.get_dummies(predict_date)
predict_date.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,14,0.0,0,1,0,0,1,0,1,0
2,5.0,1.0,39,0.0,0,0,1,1,0,0,0,1
3,6.0,1.0,19,0.0,0,0,1,1,0,0,0,1
5,10.0,1.0,2,0.0,0,0,1,1,0,0,0,1
6,6.0,1.0,4,0.0,0,0,1,0,1,0,0,1


In [17]:
del predict_date['campaign_name_通常']
del predict_date['class_name_ナイト']
del predict_date['gender_M']
predict_date.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,14,0.0,0,1,0,1,1
2,5.0,1.0,39,0.0,0,0,1,0,0
3,6.0,1.0,19,0.0,0,0,1,0,0
5,10.0,1.0,2,0.0,0,0,1,0,0
6,6.0,1.0,4,0.0,0,0,0,1,0


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [18]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_date.loc[predict_date['is_deleted'] ==1]
conti = predict_date.loc[predict_date['is_deleted'] == 0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train , y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1.
 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 1.
 1. 1. 0. 0. 1. 0. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 0.
 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1.
 1. 0. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0.
 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1.

### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

### ノック49：モデルに寄与している変数を確認しよう

### ノック50：顧客の退会を予測しよう